Import Panda und SQLalchemy

In [16]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
import psycopg2
import numpy as np

from update import update_fkz
from update import update_group
from update import update_sql_table

csv-File importieren.

In [2]:
df_bln = pd.read_csv("data/Berlin_22-23.csv", delimiter= ",", encoding= 'utf-8')

In [3]:
df_bln.head()

,ID,Typ,Bezeichnung,Bereich,Bereichsbezeichnung,Einzelplan,Einzelplanbezeichnung,Kapitel,Kapitelbezeichnung,Hauptgruppe,...,Oberfunktion,Oberfunktionsbezeichnung,Funktion,Funktionsbezeichnung,Titelart,Titel,Titelbezeichnung,Jahr,BetragTyp,Betrag
0,1,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11201,"Geldstrafen, Geldbußen, Verwarnungs- und Zwang...",2022,Soll,1000
1,2,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11906,Ersatz von Fernmeldegebühren,2022,Soll,1000
2,3,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11961,Erstattung von Steuerbeträgen,2022,Soll,1000
3,4,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11979,Verschiedene Einnahmen,2022,Soll,10000
4,5,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,12401,"Mieten für Grundstücke, Gebäude und Räume",2022,Soll,35000


Columns names anpassen, Sonderzeichen entfernen

In [4]:
df_bln.columns = df_bln.columns.str.replace(" ", "_")
df_bln.columns = df_bln.columns.str.lower()

Drop der Columns, die nicht gebraucht werden

In [5]:
df_bln = df_bln.drop(['id','typ', 'bezeichnung', 'bereich', 'bereichsbezeichnung', 
                    'hauptgruppe','hauptgruppenbezeichnung', 'obergruppe', 'obergruppenbezeichnung',
                    'oberfunktion', 'oberfunktionsbezeichnung', 'hauptfunktion', 
                    'hauptfunktionsbezeichnung'
], axis=1)

Values ins passende Format setzten und leading-0 einfügen, wo sie automatisch entfernt wurde

In [6]:
df_bln['einzelplan'] = df_bln['einzelplan'].astype(str)
df_bln['kapitel'] = df_bln['kapitel'].astype(str)
df_bln['funktion'] = df_bln['funktion'].astype(str)
df_bln['titel'] = df_bln['titel'].astype(str)

df_bln['titel'] = df_bln['titel'].apply(lambda x:x.zfill(5))
df_bln['einzelplan'] = df_bln['einzelplan'].apply(lambda x:x.zfill(2))
df_bln['kapitel'] = df_bln['kapitel'].str[-2:]
df_bln['kapitel'] = df_bln['kapitel'].apply(lambda x:x.zfill(3))
df_bln['fkz'] = df_bln['funktion'].apply(lambda x:x.zfill(3))
df_bln.head()

,einzelplan,einzelplanbezeichnung,kapitel,kapitelbezeichnung,gruppe,gruppenbezeichnung,funktion,funktionsbezeichnung,titelart,titel,titelbezeichnung,jahr,betragtyp,betrag,fkz
0,01,Abgeordnetenhaus,000,Abgeordnetenhaus,112,"Geldstrafen, Geldbußen und Zwangsgelder (einsc...",11,Politische Führung,Einnahmetitel,11201,"Geldstrafen, Geldbußen, Verwarnungs- und Zwang...",2022,Soll,1000,011
1,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,11,Politische Führung,Einnahmetitel,11906,Ersatz von Fernmeldegebühren,2022,Soll,1000,011
2,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,11,Politische Führung,Einnahmetitel,11961,Erstattung von Steuerbeträgen,2022,Soll,1000,011
3,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,11,Politische Führung,Einnahmetitel,11979,Verschiedene Einnahmen,2022,Soll,10000,011
4,01,Abgeordnetenhaus,000,Abgeordnetenhaus,124,Mieten und Pachten,11,Politische Führung,Einnahmetitel,12401,"Mieten für Grundstücke, Gebäude und Räume",2022,Soll,35000,011


Titel in gruppe und counter gesplittet, gruppe und counter eingefügt

In [7]:
df_bln["gruppe"] = df_bln["titel"].str[:3]
df_bln["counter"] = df_bln["titel"].str[3:]
df_bln

,einzelplan,einzelplanbezeichnung,kapitel,kapitelbezeichnung,gruppe,gruppenbezeichnung,funktion,funktionsbezeichnung,titelart,titel,titelbezeichnung,jahr,betragtyp,betrag,fkz,counter
0,01,Abgeordnetenhaus,000,Abgeordnetenhaus,112,"Geldstrafen, Geldbußen und Zwangsgelder (einsc...",11,Politische Führung,Einnahmetitel,11201,"Geldstrafen, Geldbußen, Verwarnungs- und Zwang...",2022,Soll,1000,011,01
1,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,11,Politische Führung,Einnahmetitel,11906,Ersatz von Fernmeldegebühren,2022,Soll,1000,011,06
2,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,11,Politische Führung,Einnahmetitel,11961,Erstattung von Steuerbeträgen,2022,Soll,1000,011,61
3,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,11,Politische Führung,Einnahmetitel,11979,Verschiedene Einnahmen,2022,Soll,10000,011,79
4,01,Abgeordnetenhaus,000,Abgeordnetenhaus,124,Mieten und Pachten,11,Politische Führung,Einnahmetitel,12401,"Mieten für Grundstücke, Gebäude und Räume",2022,Soll,35000,011,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46830,45,Allgemeine Finanzangelegenheiten,020,Stiftungen (ohne Heime),517,"Bewirtschaftung der Grundstücke, Gebäude und R...",813,Sondervermögen,Ausgabetitel,51790,"Bewirtschaftung der Grundstücke, Gebäude und R...",2023,Soll,2600,813,90
46831,45,Allgemeine Finanzangelegenheiten,020,Stiftungen (ohne Heime),519,Unterhaltung der Grundstücke und baulichen Anl...,813,Sondervermögen,Ausgabetitel,51990,Unterhaltung der Grundstücke und baulichen Anl...,2023,Soll,6500,813,90
46832,45,Allgemeine Finanzangelegenheiten,020,Stiftungen (ohne Heime),831,Erwerb von Beteiligungen und dgl. im Inland,812,Kapitalvermögen,Ausgabetitel,83190,Kauf von Wertpapieren aus zweckgebundenen Einn...,2023,Soll,1000,812,90
46833,45,Allgemeine Finanzangelegenheiten,020,Stiftungen (ohne Heime),919,Zuführungen an sonstige Rücklagen,850,Rücklagen,Ausgabetitel,91990,Zweckgebundene Zuführung an den Geldbestand,2023,Soll,1000,850,90


jahr vorhanden -  state_id eingefügt

In [8]:

df_bln["state_id"] =  3



anhand der titelart einnahmen und ausgaben in den Zahlen definiert (ginge auch mit gruppe)
titel gedropt, da redundant

In [9]:

df_bln.loc[df_bln['titelart'] == 'Einnahmetitel', 'amount'] = df_bln['betrag']
df_bln.loc[df_bln['titelart'] == 'Ausgabetitel', 'amount'] = -df_bln['betrag']

df_bln = df_bln.drop(['titel','titelart', 'betragtyp'], axis=1)
df_bln = df_bln.drop(['funktion'], axis=1)


einmal angucken, da schön

In [10]:
df_bln.head(5)

,einzelplan,einzelplanbezeichnung,kapitel,kapitelbezeichnung,gruppe,gruppenbezeichnung,funktionsbezeichnung,titelbezeichnung,jahr,betrag,fkz,counter,state_id,amount
0,01,Abgeordnetenhaus,000,Abgeordnetenhaus,112,"Geldstrafen, Geldbußen und Zwangsgelder (einsc...",Politische Führung,"Geldstrafen, Geldbußen, Verwarnungs- und Zwang...",2022,1000,011,01,3,1000.0
1,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,Politische Führung,Ersatz von Fernmeldegebühren,2022,1000,011,06,3,1000.0
2,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,Politische Führung,Erstattung von Steuerbeträgen,2022,1000,011,61,3,1000.0
3,01,Abgeordnetenhaus,000,Abgeordnetenhaus,119,Sonstige Verwaltungseinnahmen,Politische Führung,Verschiedene Einnahmen,2022,10000,011,79,3,10000.0
4,01,Abgeordnetenhaus,000,Abgeordnetenhaus,124,Mieten und Pachten,Politische Führung,"Mieten für Grundstücke, Gebäude und Räume",2022,35000,011,01,3,35000.0


Umbenennen von spalten für sql

In [11]:
df_bln.rename(columns = {'einzelplan' : 'ep', 'titelbezeichnung' : 'zweckbestimmung', 
                        'jahr' : 'year', 'kapitelbezeichnung' : 'kapitel_txt', 
                        'einzelplanbezeichnung' : 'ep_txt', 'gruppenbezeichnung' : 'gruppe_txt',
                        'funktionsbezeichnung' : 'fkz_txt'
                        }, inplace = True)

df_budget für den Import in budget_all aufbereiten

In [12]:
df_budget = df_bln[['ep', 'kapitel', 'fkz', 'zweckbestimmung', 'amount', 
'gruppe', 'counter', 'year', 'state_id']]



df_ep für den Import in einzelplaene aufbereiten

In [13]:

df_ep = df_bln.drop_duplicates(['ep', 'year'])
df_ep = df_ep[['ep', 'ep_txt', 'year', 'state_id']]
df_ep

,ep,ep_txt,year,state_id
0,01,Abgeordnetenhaus,2022,3
57,02,Verfassungsgerichtshof,2022,3
80,03,Regierende Bürgermeisterin/ Regierender Bürger...,2022,3
171,05,"Inneres, Digitalisierung und Sport",2022,3
1048,06,"Justiz, Vielfalt und Antidiskriminierung",2022,3
...,...,...,...,...
31494,40,Jugendamt,2023,3
31659,41,Gesundheitsamt,2023,3
31718,42,Stadtentwicklungsamt,2023,3
31776,43,Umwelt- und Naturschutzamt,2023,3


df_kapitel für den Import in kapitel aufbereiten

In [14]:
df_kapitel = df_bln.drop_duplicates(['ep', 'kapitel', 'year'])
df_kapitel = df_kapitel[['ep', 'kapitel', 'kapitel_txt', 'year', 'state_id']]

In [15]:
df_kapitel

,ep,kapitel,kapitel_txt,year,state_id
0,01,000,Abgeordnetenhaus,2022,3
57,02,000,Verfassungsgerichtshof,2022,3
80,03,000,Senatskanzlei,2022,3
169,03,009,Senatskanzlei - Personalüberhang -,2022,3
171,05,000,"Senatsverwaltung für Inneres, Digitalisierung ...",2022,3
...,...,...,...,...,...
36142,36,011,Geschäftsstelle Servicezentrum der Volkshochsc...,2023,3
38394,40,082,Jugendausbildungszentrum,2023,3
38948,35,081,Zentrales Fundbüro,2023,3
40307,33,082,Geschäftsstelle Schulbauoffensive der Berliner...,2023,3


Wir bereiten alles für den Push in die Database vor.

In [22]:
get_sql_config()
engine = get_engine()

Puschen

In [23]:
#push_to_database(df_budget, "budget_all", engine , "capstone_public_budgeting")

The budget_all table was imported successfully.


In [25]:
table_name = 'einzelplaene'
update_sql_table(df_ep,table_name)
#push_to_database(df_ep, "einzelplaene", engine , "capstone_public_budgeting")

The einzelplaene table was imported successfully.


In [18]:
table_name = 'kapitel'
update_sql_table(df_kapitel,table_name)
#push_to_database(df_kapitel, "kapitel", engine , "capstone_public_budgeting")

The kapitel table was imported successfully.


Gruppe und Funktionsbezeichnungen checken

In [30]:
df_gruppe = df_bln.drop_duplicates(['gruppe', 'gruppe_txt'])
df_gruppe = df_gruppe[['gruppe', 'gruppe_txt']]
update_group(df_gruppe)

The gruppen table was imported successfully.


In [31]:
df_funktion = df_bln.drop_duplicates(['fkz', 'fkz_txt'])
df_funktion = df_funktion[['fkz', 'fkz_txt']]
update_fkz(df_funktion)

The funktionsbezeichnungen table was imported successfully.
